# Deep Neural Network

Source: https://github.com/haradai1262/CIKM2020-AnalytiCup

TO DO
- le nN

In [34]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sns as sns
from sklearn.metrics import mean_absolute_error
from verstack.stratified_continuous_split import scsplit  # pip install verstack


# Load the training data
train_data = pd.read_csv("data/train.csv")
train_data.drop(['mentions'], axis=1)

# Load the evaluation data
eval_data = pd.read_csv("data/evaluation.csv")
eval_data.drop(['mentions'], axis=1)

# split data
X_train, X_test, y_train, y_test = scsplit(
    train_data, train_data['retweets_count'], stratify=train_data['retweets_count'], train_size=0.8, test_size=0.2)
# We remove the actual number of retweets from our features since it is the value that we are trying to predict
X_train = X_train.drop(['retweets_count'], axis=1)
X_test = X_test.drop(['retweets_count'], axis=1)

# Extrac features


## Extractions

## time


In [35]:
from datetime import datetime
def extract_time_features(df):
    rs_df = df
    rs_df["hour"] = rs_df['timestamp'].apply(
        lambda t: (datetime.fromtimestamp(t//1000).hour))
    rs_df["day"] = rs_df['timestamp'].apply(
        lambda t: (datetime.fromtimestamp(t//1000)).weekday())
    rs_df["week_in_month"] = rs_df['timestamp'].apply(
        lambda t: (datetime.fromtimestamp(t//1000).day)//7)  
    rs_df=rs_df.drop(['timestamp'], axis=1)
    return rs_df

## Num

In [36]:

def extract_ratio_features(df):
    rs_df = df
    rs_df['followers__favorites'] = rs_df['followers_count'] * rs_df['favorites_count']
    rs_df['friends__favorites'] = rs_df['friends_count'] * rs_df['favorites_count']
    rs_df['followers__friends__favorites'] = rs_df['followers_count'] * rs_df['friends_count'] * rs_df['favorites_count']
    return rs_df

from scipy.stats import norm
from scipy.stats import zscore
def extract_transfo(df,columns):
    rs_df = df
    for col in columns:
        mean = rs_df[col].mean()
        std = rs_df[col].std()
        rs_df[col+'_cdf'] = norm.cdf(rs_df[col].values, loc=mean, scale=std)
        rs_df[col+'_z'] = zscore(rs_df[col].values)       
        rs_df[col+'_rank'] = rs_df[col].rank(method='min')
        rs_df[col+'log'] = (df[col] + 1).apply(np.log)
    return rs_df
    

## Text

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from nltk.corpus import stopwords
def extract_topic(df):
    rs_df = df
    rs_df['hashtags'] = rs_df['hashtags'].apply(
        lambda x: x.replace('[', '').replace(']', '').replace("'", ''))
    #join text and hashtags
    rs_df['total_text'] = rs_df['text'] + ' ' + rs_df['hashtags']
    vectorizer = TfidfVectorizer(min_df=1, max_features=None, stop_words=stopwords.words('french'))
    vector = vectorizer.fit_transform(X_train['text'])
    svd = TruncatedSVD(n_components=5, n_iter=7, random_state=42)
    svd.fit(vector)
    topic=svd.transform(vector)
    rs_df['topic_1'] = topic[:,0]
    rs_df['topic_2'] = topic[:,1]
    rs_df['topic_3'] = topic[:,2]
    rs_df['topic_4'] = topic[:,3]
    rs_df['topic_5'] = topic[:,4]
    rs_df=rs_df.drop(['hashtags'],axis=1)
    rs_df=rs_df.drop(['total_text'],axis=1)
    return rs_df

from textblob import TextBlob  # pip install textblob-fr
from textblob_fr import PatternTagger, PatternAnalyzer


def sent_engineering(in_df):
    rs_df = in_df
    # add columns related to sentiment analysis
    rs_df['polarity'] = rs_df['text'].apply(lambda x: TextBlob(
        x, pos_tagger=PatternTagger(), analyzer=PatternAnalyzer()).sentiment[0])
    rs_df['subjectivity'] = rs_df['text'].apply(lambda x: TextBlob(
        x, pos_tagger=PatternTagger(), analyzer=PatternAnalyzer()).sentiment[1])
    # drop the text column
    rs_df = rs_df.drop(['text'], axis=1)

    return rs_df

def extract_url(in_df):
    #count url
    rs_df = in_df
    rs_df['url_count'] = rs_df['urls'].apply(lambda x: len(x.split(',')))
    rs_df=rs_df.drop(['urls'],axis=1)

    return rs_df


In [38]:
def extract_cluster(df,columns):
    rs_df = df
    rs_df['cluster'] = KMeans(n_clusters=100, random_state=0).fit_predict(rs_df[columns].values.reshape(-1, 1))
    return rs_df

In [39]:
tweet_metrics_features = [
    'followers_count', 'friends_count', 'favorites_count',
    'followers__favorites', 'friends__favorites', 'followers__friends__favorites',
]

tweet_metrics_log_features = [feat+'_log' for feat in tweet_metrics_features]
tweet_metrics_cdf_features = [feat+'_cdf' for feat in tweet_metrics_features]
tweet_metrics_z_features = [feat+'_z' for feat in tweet_metrics_features]
tweet_metrics_rank_features = [feat+'_rank' for feat in tweet_metrics_features]

time_cat_features = ['hour', 'day', 'week_of_month']

In [40]:
new_X_train=extract_time_features(X_train)
new_X_train=extract_ratio_features(new_X_train)
new_X_train=extract_transfo(new_X_train,tweet_metrics_features)
new_X_train=extract_topic(new_X_train)
new_X_train=sent_engineering(new_X_train)
new_X_train=extract_url(new_X_train)


            

        

In [41]:
print(new_X_train.head())


        favorites_count  followers_count  statuses_count  friends_count  \
319441             1316            18232           66732           2956   
148273                0               71            4332            245   
35324                10             3606          236301            236   
150236                2              632          213431            969   
148734               49              461            1769            293   

       mentions  verified  TweetID  hour  day  week_in_month  ...  \
319441       []         0   582659    11    2              1  ...   
148273       []         0   310171    10    1              2  ...   
35324        []         0   231722    19    4              1  ...   
150236       []         0   179476    10    5              1  ...   
148734       []         0   595839    21    2              1  ...   

        followers__friends__favorites_rank  followers__friends__favoriteslog  \
319441                            280691.0            

In [42]:
dense_features = []
dense_features += tweet_metrics_features
dense_features += tweet_metrics_log_features
dense_features += tweet_metrics_cdf_features
dense_features += tweet_metrics_z_features
dense_features += tweet_metrics_rank_features
dense_features += time_num_features
dense_features += count_encording_features
dense_features += target_encording_features
dense_features += text_tfidf_features
dense_features += varlen_count_encording_features
dense_features += varlen_target_encording_features
dense_features += user_stats_features

sparse_features = []
sparse_features += sentiment_features
sparse_features += time_cat_features
sparse_features += user_clustering_features

NameError: name 'time_num_features' is not defined

# NN library


In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F


class LogCoshLoss(torch.nn.Module):
    '''
    ref: https://github.com/tuantle/regression-losses-pytorch/blob/master/regression_losses.py
    '''
    def __init__(self):
        super().__init__()

    def forward(self, y_t, y_prime_t):
        ey_t = y_t - y_prime_t
        return torch.mean(torch.log(torch.cosh(ey_t + 1e-12)))


class DNN(nn.Module):
    '''
    ref: https://github.com/shenweichen/DeepCTR-Torch/blob/master/deepctr_torch/layers/core.py
    '''
    """The Multi Layer Percetron
      Input shape
        - nD tensor with shape: ``(batch_size, ..., input_dim)``. The most common situation would be a 2D input with shape ``(batch_size, input_dim)``.
      Output shape
        - nD tensor with shape: ``(batch_size, ..., hidden_size[-1])``. For instance, for a 2D input with shape ``(batch_size, input_dim)``, the output would have shape ``(batch_size, hidden_size[-1])``.
      Arguments
        - **inputs_dim**: input feature dimension.
        - **hidden_units**:list of positive integer, the layer number and units in each layer.
        - **activation**: Activation function to use.
        - **l2_reg**: float between 0 and 1. L2 regularizer strength applied to the kernel weights matrix.
        - **dropout_rate**: float in [0,1). Fraction of the units to dropout.
        - **use_bn**: bool. Whether use BatchNormalization before activation or not.
        - **seed**: A Python integer to use as random seed.
    """

    def __init__(self, inputs_dim, hidden_units, activation='relu', l2_reg=0, dropout_rate=0, use_bn=False,
                 init_std=0.0001, dice_dim=3, seed=1024, device='cpu'):
        super(DNN, self).__init__()
        self.dropout_rate = dropout_rate
        self.dropout = nn.Dropout(dropout_rate)
        self.seed = seed
        self.l2_reg = l2_reg
        self.use_bn = use_bn
        if len(hidden_units) == 0:
            raise ValueError("hidden_units is empty!!")
        hidden_units = [inputs_dim] + list(hidden_units)

        self.linears = nn.ModuleList(
            [nn.Linear(hidden_units[i], hidden_units[i + 1]) for i in range(len(hidden_units) - 1)])

        if self.use_bn:
            self.bn = nn.ModuleList(
                [nn.BatchNorm1d(hidden_units[i + 1]) for i in range(len(hidden_units) - 1)])

        self.activation_layers = nn.ModuleList(
            [activation_layer(activation, hidden_units[i + 1], dice_dim) for i in range(len(hidden_units) - 1)])

        for name, tensor in self.linears.named_parameters():
            if 'weight' in name:
                nn.init.normal_(tensor, mean=0, std=init_std)

        self.to(device)

    def forward(self, inputs):
        deep_input = inputs

        for i in range(len(self.linears)):

            fc = self.linears[i](deep_input)

            if self.use_bn:
                fc = self.bn[i](fc)

            fc = self.activation_layers[i](fc)

            fc = self.dropout(fc)
            deep_input = fc
        return deep_input

# MLP

In [ ]:
class MLP(nn.Module):
    def __init__(
        self,
        dnn_input, dnn_hidden_units, dnn_dropout,
        activation='relu', use_bn=True, l2_reg=1e-4, init_std=1e-4,
        device='cpu',
        feature_index={},
        embedding_dict={},
        dense_features=[],
        sparse_features=[],
        varlen_sparse_features=[],
        varlen_mode_list=[],
        embedding_size=8,
        batch_size=256,
    ):
        super().__init__()
        self.device = device
        self.feature_index = feature_index
        self.embedding_dict = embedding_dict
        self.dense_features = dense_features
        self.sparse_features = sparse_features
        self.varlen_sparse_features = varlen_sparse_features
        self.varlen_mode_list = varlen_mode_list
        self.embedding_size = embedding_size
        self.batch_size = batch_size

        self.reg_loss = torch.zeros((1,), device=device)

        self.dnn = DNN(#######################################USEFUl
            dnn_input, dnn_hidden_units,
            activation='relu', l2_reg=l2_reg, dropout_rate=dnn_dropout, use_bn=use_bn,
            init_std=init_std, device=device
        )
        self.dnn_linear = nn.Linear(dnn_hidden_units[-1], 1, bias=False).to(device)

        # add regularization
        self.add_regularization_loss(self.embedding_dict.parameters(), l2_reg)
        self.add_regularization_loss(
            filter(lambda x: 'weight' in x[0] and 'bn' not in x[0], self.dnn.named_parameters()), l2_reg)
        self.add_regularization_loss(self.dnn_linear.weight, l2_reg)

        self.out = nn.Sigmoid()

        self.to(device)

    def forward(self, X):

        dense_value_list = [
            X[:, self.feature_index[feat]: self.feature_index[feat] + 1] for feat in self.dense_features
        ]
        sparse_embedding_list = [
            self.embedding_dict[feat](
                X[:, self.feature_index[feat]: self.feature_index[feat] + 1].long()
            ) for feat in self.sparse_features
        ]
        varlen_sparse_embedding_list = get_varlen_pooling_list(
            self.embedding_dict, X, self.feature_index, self.varlen_sparse_features, self.varlen_mode_list, self.device
        )
        sparse_embedding_list = sparse_embedding_list + varlen_sparse_embedding_list

        sparse_dnn_input = torch.flatten(torch.cat(sparse_embedding_list, dim=-1), start_dim=1)
        dense_dnn_input = torch.flatten(torch.cat(dense_value_list, dim=-1), start_dim=1)

        dnn_input = torch.cat([sparse_dnn_input, dense_dnn_input], dim=-1)


        dnn_output = self.dnn(dnn_input)
        
        dnn_logit = self.dnn_linear(dnn_output)

        y_pred = self.out(dnn_logit)
        return y_pred